## 04/02 ~ continuing work with py files

In [1]:
from src.data_cleaning import data_labeling
from src.data_cleaning import feature_engineering
from src.language_processing import to_tfidf

In [2]:
import pandas as pd
#pd.set_option('display.max_columns', 100)

In [3]:
df = data_labeling('data/data_prepping_zeros_index.xlsx',
              'data/smalleyeball_partone.xlsx',
              'data/smalleyeball_parttwo.xlsx')

/home/daniel/Desktop/Seattle_g89/spam_filter/look2social_spamfilter/src/data_cleaning.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['spam_corporate'][df.screen_name.isin(combined_lst)] = 1
/home/daniel/Desktop/Seattle_g89/spam_filter/look2social_spamfilter/src/data_cleaning.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['spam_bot'][df.expanded_url == 'https://acrobat.adobe.com'] = 1
/home/daniel/Desktop/Seattle_g89/spam_filter/look2social_spamfilter/src/data_cleaning.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cavea

In [4]:
df2 = feature_engineering(df)

/home/daniel/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
/home/daniel/Desktop/Seattle_g89/spam_filter/look2social_spamfilter/src/data_cleaning.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df3['time_created'] = pd.to_datetime(df3['created_at']).copy()
/home/daniel/Desktop/Seattle_g89/spam_filter/look2social_spamfilter/src/data_cleaning.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

In [5]:
df2.head()

,text,spam,spam_marketing,spam_hijack,spam_corporate,spam_bot,spam_known,spam_own,Docusign,onespan,...,adobe sign,listed_count,description,statuses_count,followers_count,favourites_count,friends_count,time_float_sin,time_float_cos,is_description_none
0,just closed a deal in 27 hours using #AdobeSig...,0,0,0,0,1,0,0,False,False,...,True,1,There's no substitute for experience! Call 609...,6750,108,1,12,-0.049140,0.998792,False
1,just closed a deal in 2 days using #AdobeSign ...,0,0,0,0,1,0,0,False,False,...,True,14,We provide an affordable way to have a wonderf...,5018,47,2,25,-0.501007,0.865443,False
2,just closed a deal in 2 hours using #AdobeSign...,0,0,0,0,1,0,0,False,False,...,True,0,,29,60,10,52,-0.599548,0.800339,True
3,just closed a deal in 26 hours using #AdobeSig...,0,0,0,0,1,0,0,False,False,...,True,2,,5332,66,0,182,-0.325843,0.945424,True
4,just closed a deal in 6 days using #AdobeSign ...,0,0,0,0,1,0,0,False,False,...,True,8,Carefully Crafted & Intentionally Inspired Eve...,1337,225,19,348,-0.098162,0.995170,False


In [6]:
df = df2

In [7]:
text_tfidf_matrix = to_tfidf(df,'text')

In [8]:
desc_tfidf_matrix = to_tfidf(df,'description')

In [8]:
text_tfidf_matrix.mean()

0.00022003398980305908

In [9]:
desc_tfidf_matrix.mean()

0.000551184644606618

# Finally starting modeling!

## Approaching way

1. numerical predictors will be modeled with logistic regression, random forest, gradient boosting

2. TFIDF matrix predictors will be modeled by Naive Bayes. Currently, as I'm testing with local computer, I reduced down the number of max features to 1000. However,if I start cloud computing (Amazon AWS), I will increase the number of max features

3. Ultimately, I will ensemble three different models - one based on numerical predictors and two based on post text TFIDF matrix and description text TFIDF matrix. After evaluating all model, I will find a best ratio for the weight of each model to be ensembled.

## Spam_bot

### Logistic Regression

In [9]:
%matplotlib inline
import matplotlib.pyplot as plt
from scipy import stats
import scipy
import numpy as np
from sklearn.linear_model import LogisticRegression

In [10]:
from sklearn.model_selection import train_test_split

spam bot is the minimum level of spam to work with. I will start with this one.

In [12]:
df.columns

Index(['text', 'spam', 'spam_marketing', 'spam_hijack', 'spam_corporate',
       'spam_bot', 'spam_known', 'spam_own', 'Docusign', 'onespan', 'signnow',
       'adobe sign', 'listed_count', 'description', 'statuses_count',
       'followers_count', 'favourites_count', 'friends_count',
       'time_float_sin', 'time_float_cos', 'is_description_none',
       'text_without_accent', 'text_word_tokens', 'processed_text', 'stemmed',
       'ngrammed'],
      dtype='object')

In [11]:
df_model_training = df.iloc[:16000]
df_final_validation = df.iloc[16000:]

In [21]:
y = df_model_training['spam_bot']
X = df_model_training[['Docusign', 'onespan', 'signnow',
       'adobe sign', 'listed_count', 'statuses_count',
       'followers_count', 'favourites_count', 'friends_count',
       'time_float_sin', 'time_float_cos', 'is_description_none']]

In [22]:
X.head()

,Docusign,onespan,signnow,adobe sign,listed_count,statuses_count,followers_count,favourites_count,friends_count,time_float_sin,time_float_cos,is_description_none
0,False,False,False,True,1,6750,108,1,12,-0.049140,0.998792,False
1,False,False,False,True,14,5018,47,2,25,-0.501007,0.865443,False
2,False,False,False,True,0,29,60,10,52,-0.599548,0.800339,True
3,False,False,False,True,2,5332,66,0,182,-0.325843,0.945424,True
4,False,False,False,True,8,1337,225,19,348,-0.098162,0.995170,False


In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [26]:
logistic_model = LogisticRegression()

In [27]:
logistic_model.fit(X_train,y_train)

/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [29]:
y_predict = logistic_model.predict(X_test)

In [63]:
y_pred = logistic_model.predict_proba(X_test)

In [61]:
sum(y) / len(y)

0.727125

In [72]:
np.argmax(logistic_model.coef_)
X.columns[3]

'adobe sign'

In [64]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import log_loss
from sklearn.metrics import roc_auc_score

print(accuracy_score(y_test,y_predict))
print(confusion_matrix(y_test,y_predict))
print(log_loss(y_test,y_pred))
print(roc_auc_score(y_test,y_predict))

0.98825
[[1010   47]
 [   0 2943]]
0.11483660686215527
0.9777672658467361


In [16]:
from sklearn.ensemble import RandomForestClassifier

In [45]:
rf_model = RandomForestClassifier()

In [46]:
rf_model.fit(X_train,y_train)

/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [47]:
y_predict = logistic_model.predict(X_test)

In [66]:
y_pred_rf = rf_model.predict_proba(X_test)

In [67]:
print(accuracy_score(y_test,y_predict))
print(confusion_matrix(y_test,y_predict))
print(log_loss(y_test,y_pred_rf))
print(roc_auc_score(y_test,y_predict))

0.98825
[[1010   47]
 [   0 2943]]
0.12788181628188067
0.9777672658467361


In [19]:
from sklearn.ensemble import GradientBoostingClassifier

In [18]:
gb_model = GradientBoostingClassifier()

NameError: name 'GradientBoostingClassifier' is not defined

In [54]:
gb_model.fit(X_train,y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              n_iter_no_change=None, presort='auto', random_state=None,
              subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)

In [55]:
y_predict = gb_model.predict(X_test)

In [68]:
y_pred_gb = gb_model.predict_proba(X_test)

In [69]:
print(accuracy_score(y_test,y_predict))
print(confusion_matrix(y_test,y_predict))
print(log_loss(y_test,y_pred_gb))
print(roc_auc_score(y_test,y_predict))

0.98825
[[1010   47]
 [   0 2943]]
0.04897810445502796
0.9777672658467361


This is understandable performance - every automation post is from Adobe Sign and there is a boolean column to see if the post is about Adobe Sign or not.

As the task was easy, good performance was understandable.

In [73]:
X.columns[np.argmax(logistic_model.coef_)]

'adobe sign'

In [77]:
test_logistic_regressio(df_model_training, 'spam_bot')

/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Accuracy score of this model is

0.873

Confusion Matrix of this model is

[[ 640  450]
 [  58 2852]]

Log-Loss score of this model is

0.2840825953234961

AUC score of this model is

0.7836123459125446


In [ ]:
def test_model(data,column,model):
    y = data[column]
    X = data[['Docusign', 'onespan', 'signnow','adobe sign',
                           'listed_count', 'statuses_count','followers_count',
                           'favourites_count', 'friends_count','time_float_sin',
                           'time_float_cos', 'is_description_none']]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
    model = model()
    model.fit(X_train,y_train)
    y_predict = model.predict(X_test)
    y_pred = model.predict_proba(X_test)

    print("Accuracy score of this model is" + "\n")
    print(accuracy_score(y_test,y_predict))
    print("\n" + "Confusion Matrix of this model is" + "\n")
    print(confusion_matrix(y_test,y_predict))
    print("\n" + "Log-Loss score of this model is" + "\n")
    print(log_loss(y_test,y_pred))
    print("\n" + "AUC score of this model is" + "\n")
    print(roc_auc_score(y_test,y_predict))
    return None

In [13]:
from src.modeling import test_model

In [35]:
test_model(df_model_training, 'spam_bot',LogisticRegression)

/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Accuracy score of this model is

0.9545

Confusion Matrix of this model is

[[ 910  164]
 [  18 2908]]

Log-Loss score of this model is

0.1601974900558897

AUC score of this model is

0.9205740353932064


In [36]:
test_model(df_model_training, 'spam_bot',RandomForestClassifier)

/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Accuracy score of this model is

0.988

Confusion Matrix of this model is

[[1057   39]
 [   9 2895]]

Log-Loss score of this model is

0.12027931096985112

AUC score of this model is

0.9806584424202206


In [38]:
test_model(df_model_training, 'spam_bot',GradientBoostingClassifier)

Accuracy score of this model is

0.987

Confusion Matrix of this model is

[[1025   52]
 [   0 2923]]

Log-Loss score of this model is

0.055195422549462324

AUC score of this model is

0.9758588672237698


In [14]:
test_model(df_model_training, 'spam_corporate',LogisticRegression)

/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Accuracy score of this model is

0.9865

Confusion Matrix of this model is

[[3805   12]
 [  42  141]]

Log-Loss score of this model is

0.08365115046233867

AUC score of this model is

0.8836739865227605


In [17]:
test_model(df_model_training, 'spam_corporate',RandomForestClassifier)

/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Accuracy score of this model is

0.99875

Confusion Matrix of this model is

[[3826    0]
 [   5  169]]

Log-Loss score of this model is

0.005499953878869179

AUC score of this model is

0.985632183908046


In [20]:
test_model(df_model_training, 'spam_corporate',GradientBoostingClassifier)

Accuracy score of this model is

0.998

Confusion Matrix of this model is

[[3840    3]
 [   5  152]]

Log-Loss score of this model is

0.01210257704747622

AUC score of this model is

0.9836861130585679


In [22]:
test_model(df_model_training, 'spam_marketing',LogisticRegression)

/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Accuracy score of this model is

0.9525

Confusion Matrix of this model is

[[3810    6]
 [ 184    0]]

Log-Loss score of this model is

0.1593836533541995

AUC score of this model is

0.4992138364779874


In [23]:
test_model(df_model_training, 'spam_marketing',RandomForestClassifier)

/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Accuracy score of this model is

0.96625

Confusion Matrix of this model is

[[3793   38]
 [  97   72]]

Log-Loss score of this model is

0.2956174728790855

AUC score of this model is

0.7080582108893656


In [24]:
test_model(df_model_training, 'spam_marketing',GradientBoostingClassifier)

Accuracy score of this model is

0.95925

Confusion Matrix of this model is

[[3809   11]
 [ 152   28]]

Log-Loss score of this model is

0.11175823866939387

AUC score of this model is

0.5763379872018615


In [25]:
test_model(df_model_training, 'spam_hijack',LogisticRegression)

/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Accuracy score of this model is

0.87925

Confusion Matrix of this model is

[[3298   44]
 [ 439  219]]

Log-Loss score of this model is

0.25328555461820684

AUC score of this model is

0.6598304893598833


In [26]:
test_model(df_model_training, 'spam_hijack', RandomForestClassifier)

/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Accuracy score of this model is

0.94225

Confusion Matrix of this model is

[[3188  126]
 [ 105  581]]

Log-Loss score of this model is

0.3367466455682694

AUC score of this model is

0.9044591282499724


In [27]:
test_model(df_model_training, 'spam_hijack', GradientBoostingClassifier)

Accuracy score of this model is

0.9445

Confusion Matrix of this model is

[[3149  176]
 [  46  629]]

Log-Loss score of this model is

0.13284233319922711

AUC score of this model is

0.939459760512392


In [29]:
df

,text,spam,spam_marketing,spam_hijack,spam_corporate,spam_bot,spam_known,spam_own,Docusign,onespan,...,favourites_count,friends_count,time_float_sin,time_float_cos,is_description_none,text_without_accent,text_word_tokens,processed_text,stemmed,ngrammed
0,just closed a deal in 27 hours using #AdobeSig...,0,0,0,0,1,0,0,False,False,...,1,12,-0.049140,0.998792,False,There's no substitute for experience! Call 609...,"[There, 's, no, substitute, for, experience, !...","[Theres, substitute, experience, Call, 6094028...","[there, substitut, experi, call, 6094028900, t...","[there, substitut, experi, call, 6094028900, t..."
1,just closed a deal in 2 days using #AdobeSign ...,0,0,0,0,1,0,0,False,False,...,2,25,-0.501007,0.865443,False,We provide an affordable way to have a wonderf...,"[We, provide, an, affordable, way, to, have, a...","[provide, affordable, way, wonderful, characte...","[provid, afford, way, wonder, charact, attend,...","[provid, afford, way, wonder, charact, attend,..."
2,just closed a deal in 2 hours using #AdobeSign...,0,0,0,0,1,0,0,False,False,...,10,52,-0.599548,0.800339,True,,[],[],[],[]
3,just closed a deal in 26 hours using #AdobeSig...,0,0,0,0,1,0,0,False,False,...,0,182,-0.325843,0.945424,True,,[],[],[],[]
4,just closed a deal in 6 days using #AdobeSign ...,0,0,0,0,1,0,0,False,False,...,19,348,-0.098162,0.995170,False,Carefully Crafted & Intentionally Inspired Eve...,"[Carefully, Crafted, &, Intentionally, Inspire...","[Carefully, Crafted, Intentionally, Inspired, ...","[care, craft, intent, inspir, event, plan, vin...","[care, craft, intent, inspir, event, plan, vin..."
5,just closed a deal in 35 minutes using #AdobeS...,0,0,0,0,1,0,0,False,False,...,0,182,-0.460587,0.887615,True,,[],[],[],[]
6,just closed a deal in 3 days using #AdobeSign ...,0,0,0,0,1,0,0,False,False,...,0,87,-0.305003,0.952351,False,SP Guard from Iconix uses enhancements to stan...,"[SP, Guard, from, Iconix, uses, enhancements, ...","[SP, Guard, Iconix, uses, enhancements, standa...","[SP, guard, iconix, use, enhanc, standardsbas,...","[SP, guard, iconix, use, enhanc, standardsbas,..."
7,just closed a deal in 58 minutes using #AdobeS...,0,0,0,0,1,0,0,False,False,...,4,17,-0.462587,0.886574,True,,[],[],[],[]
8,just closed a deal in 7 minutes using #AdobeSi...,0,0,0,0,1,0,0,False,False,...,14184,84426,-0.481626,0.876377,False,"Social Media Consultant, Real estate investor ...","[Social, Media, Consultant, ,, Real, estate, i...","[Social, Media, Consultant, Real, estate, inve...","[social, media, consult, real, estat, investor...","[social, media, consult, real, estat, investor..."
9,So apparently IU is catching on and wonвЂ™t ju...,0,0,0,0,0,0,0,False,False,...,15151,109,-0.673604,0.739092,False,BSU 17 - SIU 19 // Architecture & Interior Des...,"[BSU, 17, -, SIU, 19, //, Architecture, &, Int...","[BSU, 17, SIU, 19, Architecture, Interior, Des...","[bsu, 17, siu, 19, architectur, interior, desi...","[bsu, 17, siu, 19, architectur, interior, desi..."


In [30]:
df3 = df

In [33]:
df3['spam'][(df3['spam_bot']|df3['spam_corporate']==1)]=1

/home/daniel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [34]:
df3

,text,spam,spam_marketing,spam_hijack,spam_corporate,spam_bot,spam_known,spam_own,Docusign,onespan,...,favourites_count,friends_count,time_float_sin,time_float_cos,is_description_none,text_without_accent,text_word_tokens,processed_text,stemmed,ngrammed
0,just closed a deal in 27 hours using #AdobeSig...,1,0,0,0,1,0,0,False,False,...,1,12,-0.049140,0.998792,False,There's no substitute for experience! Call 609...,"[There, 's, no, substitute, for, experience, !...","[Theres, substitute, experience, Call, 6094028...","[there, substitut, experi, call, 6094028900, t...","[there, substitut, experi, call, 6094028900, t..."
1,just closed a deal in 2 days using #AdobeSign ...,1,0,0,0,1,0,0,False,False,...,2,25,-0.501007,0.865443,False,We provide an affordable way to have a wonderf...,"[We, provide, an, affordable, way, to, have, a...","[provide, affordable, way, wonderful, characte...","[provid, afford, way, wonder, charact, attend,...","[provid, afford, way, wonder, charact, attend,..."
2,just closed a deal in 2 hours using #AdobeSign...,1,0,0,0,1,0,0,False,False,...,10,52,-0.599548,0.800339,True,,[],[],[],[]
3,just closed a deal in 26 hours using #AdobeSig...,1,0,0,0,1,0,0,False,False,...,0,182,-0.325843,0.945424,True,,[],[],[],[]
4,just closed a deal in 6 days using #AdobeSign ...,1,0,0,0,1,0,0,False,False,...,19,348,-0.098162,0.995170,False,Carefully Crafted & Intentionally Inspired Eve...,"[Carefully, Crafted, &, Intentionally, Inspire...","[Carefully, Crafted, Intentionally, Inspired, ...","[care, craft, intent, inspir, event, plan, vin...","[care, craft, intent, inspir, event, plan, vin..."
5,just closed a deal in 35 minutes using #AdobeS...,1,0,0,0,1,0,0,False,False,...,0,182,-0.460587,0.887615,True,,[],[],[],[]
6,just closed a deal in 3 days using #AdobeSign ...,1,0,0,0,1,0,0,False,False,...,0,87,-0.305003,0.952351,False,SP Guard from Iconix uses enhancements to stan...,"[SP, Guard, from, Iconix, uses, enhancements, ...","[SP, Guard, Iconix, uses, enhancements, standa...","[SP, guard, iconix, use, enhanc, standardsbas,...","[SP, guard, iconix, use, enhanc, standardsbas,..."
7,just closed a deal in 58 minutes using #AdobeS...,1,0,0,0,1,0,0,False,False,...,4,17,-0.462587,0.886574,True,,[],[],[],[]
8,just closed a deal in 7 minutes using #AdobeSi...,1,0,0,0,1,0,0,False,False,...,14184,84426,-0.481626,0.876377,False,"Social Media Consultant, Real estate investor ...","[Social, Media, Consultant, ,, Real, estate, i...","[Social, Media, Consultant, Real, estate, inve...","[social, media, consult, real, estat, investor...","[social, media, consult, real, estat, investor..."
9,So apparently IU is catching on and wonвЂ™t ju...,0,0,0,0,0,0,0,False,False,...,15151,109,-0.673604,0.739092,False,BSU 17 - SIU 19 // Architecture & Interior Des...,"[BSU, 17, -, SIU, 19, //, Architecture, &, Int...","[BSU, 17, SIU, 19, Architecture, Interior, Des...","[bsu, 17, siu, 19, architectur, interior, desi...","[bsu, 17, siu, 19, architectur, interior, desi..."


In [39]:
df_model_training = df3.iloc[:16000]
df_final_validation = df3.iloc[16000:]

In [40]:
test_model(df_model_training, 'spam', LogisticRegression)

/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Accuracy score of this model is

0.9185

Confusion Matrix of this model is

[[ 617  274]
 [  52 3057]]

Log-Loss score of this model is

0.23693054691089793

AUC score of this model is

0.8378773619472667


In [41]:
test_model(df_model_training, 'spam', RandomForestClassifier)

/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Accuracy score of this model is

0.9895

Confusion Matrix of this model is

[[ 876   35]
 [   7 3082]]

Log-Loss score of this model is

0.1383288321425

AUC score of this model is

0.9796572875175146


In [42]:
test_model(df_model_training, 'spam', GradientBoostingClassifier)

Accuracy score of this model is

0.9825

Confusion Matrix of this model is

[[ 837   59]
 [  11 3093]]

Log-Loss score of this model is

0.06313769992192084

AUC score of this model is

0.9653039856406481


In [43]:
df3 = df
df3['spam'][(df3['spam_bot']|df3['spam_marketing']==1)]=1
df_model_training = df3.iloc[:16000]
df_final_validation = df3.iloc[16000:]

/home/daniel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [44]:
test_model(df_model_training, 'spam', LogisticRegression)

/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Accuracy score of this model is

0.914

Confusion Matrix of this model is

[[ 518  210]
 [ 134 3138]]

Log-Loss score of this model is

0.2311290538042274

AUC score of this model is

0.8352924581530939


In [45]:
test_model(df_model_training, 'spam', RandomForestClassifier)

/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Accuracy score of this model is

0.9605

Confusion Matrix of this model is

[[ 679   53]
 [ 105 3163]]

Log-Loss score of this model is

0.3355538894565465

AUC score of this model is

0.9477329427266222


In [46]:
test_model(df_model_training, 'spam', GradientBoostingClassifier)

Accuracy score of this model is

0.95125

Confusion Matrix of this model is

[[ 699   50]
 [ 145 3106]]

Log-Loss score of this model is

0.1354238121340311

AUC score of this model is

0.9443213323701571


In [47]:
df3 = df
df3['spam'][(df3['spam_bot']|df3['spam_hijack']==1)]=1
df_model_training = df3.iloc[:16000]
df_final_validation = df3.iloc[16000:]

/home/daniel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [48]:
test_model(df_model_training, 'spam', LogisticRegression)
test_model(df_model_training, 'spam', RandomForestClassifier)
test_model(df_model_training, 'spam', GradientBoostingClassifier)

/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Accuracy score of this model is

0.9555

Confusion Matrix of this model is

[[   2  168]
 [  10 3820]]

Log-Loss score of this model is

0.48309206861364273

AUC score of this model is

0.5045768699124559
Accuracy score of this model is

0.95525

Confusion Matrix of this model is

[[  68  118]
 [  61 3753]]

Log-Loss score of this model is

0.2942741692740797

AUC score of this model is

0.6747988452278251
Accuracy score of this model is

0.9555

Confusion Matrix of this model is

[[  33  156]
 [  22 3789]]

Log-Loss score of this model is

0.11701663172546066

AUC score of this model is

0.5844152057744291


In [49]:
df3 = df
df3['spam'][(df3['spam_corporate']|df3['spam_marketing']==1)]=1
df_model_training = df3.iloc[:16000]
df_final_validation = df3.iloc[16000:]

/home/daniel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [50]:
test_model(df_model_training, 'spam', LogisticRegression)
test_model(df_model_training, 'spam', RandomForestClassifier)
test_model(df_model_training, 'spam', GradientBoostingClassifier)

/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Accuracy score of this model is

0.95325

Confusion Matrix of this model is

[[   3  183]
 [   4 3810]]

Log-Loss score of this model is

0.46942472614234726

AUC score of this model is

0.5075401322800548
Accuracy score of this model is

0.95375

Confusion Matrix of this model is

[[  53  117]
 [  68 3762]]

Log-Loss score of this model is

0.28240662484331985

AUC score of this model is

0.6470050683458762
Accuracy score of this model is

0.951

Confusion Matrix of this model is

[[  19  182]
 [  14 3785]]

Log-Loss score of this model is

0.12246338682008719

AUC score of this model is

0.5454210914367358


In [52]:
df3 = df
df3['spam'][(df3['spam_corporate']|df3['spam_hijack']==1)]=1
df_model_training = df3.iloc[:16000]
df_final_validation = df3.iloc[16000:]
test_model(df_model_training, 'spam', LogisticRegression)
test_model(df_model_training, 'spam', RandomForestClassifier)
test_model(df_model_training, 'spam', GradientBoostingClassifier)

/home/daniel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Accuracy score of this model is

0.95775

Confusion Matrix of this model is

[[   2  166]
 [   3 3829]]

Log-Loss score of this model is

0.42943937688611356

AUC score of this model is

0.5055609404513371
Accuracy score of this model is

0.9575

Confusion Matrix of this model is

[[  55  127]
 [  43 3775]]

Log-Loss score of this model is

0.32863253594292496

AUC score of this model is

0.6454676805645899
Accuracy score of this model is

0.951

Confusion Matrix of this model is

[[  23  180]
 [  16 3781]]

Log-Loss score of this model is

0.12832314277803936

AUC score of this model is

0.5545433197844811


In [53]:
df3 = df
df3['spam'][(df3['spam_marketing']|df3['spam_hijack']==1)]=1
df_model_training = df3.iloc[:16000]
df_final_validation = df3.iloc[16000:]
test_model(df_model_training, 'spam', LogisticRegression)
test_model(df_model_training, 'spam', RandomForestClassifier)
test_model(df_model_training, 'spam', GradientBoostingClassifier)

/home/daniel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/daniel/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Accuracy score of this model is

0.949

Confusion Matrix of this model is

[[   1  195]
 [   9 3795]]

Log-Loss score of this model is

0.4943374856058593

AUC score of this model is

0.5013680551084787
Accuracy score of this model is

0.94825

Confusion Matrix of this model is

[[  55  145]
 [  62 3738]]

Log-Loss score of this model is

0.3821043046742817

AUC score of this model is

0.6293421052631579
Accuracy score of this model is

0.951

Confusion Matrix of this model is

[[  38  170]
 [  26 3766]]

Log-Loss score of this model is

0.12579911288168436

AUC score of this model is

0.5879178838039598
